# Day 6 Module 1: Quantum Chemistry Foundations 🌌⚛️

## ChemML 7-Day QuickStart Bootcamp - Day 6 Module 1

**Focus:** Core quantum chemistry fundamentals and Hamiltonian engineering  
**Duration:** 80-90 minutes  
**Difficulty:** ⭐⭐⭐⭐ (Advanced)

### 🎯 **Module Learning Objectives:**
1. **Master molecular Hamiltonian construction** for quantum simulation
2. **Implement Pauli string manipulation** and fermionic operators
3. **Design quantum circuits** for molecular systems
4. **Apply basis transformations** and symmetry reduction techniques
5. **Build foundation** for VQE and quantum algorithms

### 🗺️ **Module Navigation:**
- **Previous:** [Day 5 Module 3 - Production Integration](day_05_module_3_production.ipynb)
- **Current:** **Day 6 Module 1 - Quantum Chemistry Foundations** 👈
- **Next:** [Day 6 Module 2 - VQE & Molecular Ground States](day_06_module_2_vqe_algorithms.ipynb)

### 📋 **Module Contents:**
1. **Quantum Chemistry Fundamentals** - Molecular systems to quantum operators
2. **Hamiltonian Engineering** - Building molecular Hamiltonians
3. **Quantum Circuit Design** - Hardware-efficient ansätze
4. **Foundation Assessment** - Checkpoint & progress tracking

---

### ✅ **Learning Track Compatibility:**
- **🚀 Fast Track:** Focus on Sections 1-2 (Core Hamiltonian construction)
- **📚 Complete Track:** Complete all sections with full implementations
- **🎯 Flexible Track:** Modular progression with optional deep-dives

---

## 🎯 Progress Tracking & Prerequisites

### ✅ **Prerequisites Check:**
- [ ] Completed Day 5 (Quantum ML foundations)
- [ ] Basic quantum computing knowledge (qubits, gates, circuits)
- [ ] Molecular chemistry background (orbital theory, Hartree-Fock)
- [ ] Python quantum libraries (Qiskit, OpenFermion)

### 📊 **Module Progress:**
**Completion Status:** [ ] Not Started [ ] In Progress [ ] Completed

**Time Tracking:**
- Start Time: _____ 
- Target Duration: 80-90 minutes
- Actual Duration: _____

**Learning Checkpoints:**
- [ ] Molecular Hamiltonian fundamentals understood
- [ ] Hamiltonian builder implementation completed
- [ ] Quantum circuit design patterns mastered
- [ ] H2 molecule simulation successful

---

## 1️⃣ Quantum Chemistry Fundamentals & Hamiltonian Engineering ⚛️

### 🎯 **Section Objectives:**
- Build molecular Hamiltonians for quantum simulation
- Implement Pauli string manipulation and fermionic operators
- Create quantum circuits for molecular systems
- Develop basis transformation and symmetry reduction techniques

In [ ]:
# 🚀 Modern Quantum Computing Foundations with ChemML
# ====================================================

# System imports
import sys
import os
sys.path.append('../../../../src')  # Add ChemML source to path

# Core scientific libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# ChemML Modern Quantum Suite (Qiskit 2.0+ compatible)
from chemml.research.modern_quantum import (
    ModernVQE, 
    ModernQAOA,
    QuantumFeatureMap,
    MolecularHamiltonianBuilder,
    HardwareEfficientAnsatz,
    QuantumChemistryWorkflow
)

# Modern Qiskit 2.0+ imports (using primitives instead of legacy algorithms)
from qiskit import QuantumCircuit, transpile
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit.circuit import ParameterVector, Parameter

# Classical quantum chemistry for comparison
try:
    from pyscf import gto, scf
    HAS_PYSCF = True
    print("✅ PySCF available for classical comparison")
except ImportError:
    HAS_PYSCF = False
    print("⚠️ PySCF not available - quantum-only calculations")

print("✅ Modern Quantum Foundations Loaded Successfully!")
print("📡 Using Qiskit 2.0+ primitives (StatevectorEstimator/StatevectorSampler)")
print("🧬 Ready for quantum chemistry foundations!")

In [ ]:
class MolecularHamiltonianBuilder:
    """
    Build molecular Hamiltonians for quantum simulation
    """
    
    def __init__(self, molecule_config):
        self.molecule_config = molecule_config
        self.n_qubits = None
        self.hamiltonian = None
        self.fermionic_op = None
        
    def build_molecule(self, geometry, basis='sto-3g', charge=0, multiplicity=1):
        """
        Build molecular system using PySCF
        """
        self.geometry = geometry
        self.basis = basis
        
        # Create PySCF molecule
        self.mol = gto.Molecule()
        self.mol.atom = geometry
        self.mol.basis = basis
        self.mol.charge = charge
        self.mol.spin = multiplicity - 1
        self.mol.build()
        
        # Run Hartree-Fock calculation
        self.mf = scf.RHF(self.mol)
        self.mf.run()
        
        print(f"Molecule built: {len(self.mol.atom)} atoms")
        print(f"HF Energy: {self.mf.e_tot:.6f} Ha")
        
        return self
    
    def generate_hamiltonian(self, mapper='jordan_wigner', active_space=None):
        """
        Generate molecular Hamiltonian in qubit basis
        """
        # Get molecular integrals
        h1e = self.mf.mo_coeff.T @ self.mf.get_hcore() @ self.mf.mo_coeff
        h2e = ao2mo.full(self.mol, self.mf.mo_coeff)
        
        # Apply active space approximation if specified
        if active_space:
            n_active, n_electrons = active_space
            h1e = h1e[:n_active, :n_active]
            h2e = h2e[:n_active, :n_active, :n_active, :n_active]
        
        # Build fermionic Hamiltonian
        self.fermionic_op = self._build_fermionic_hamiltonian(h1e, h2e)
        
        # Transform to qubit operators
        if mapper == 'jordan_wigner':
            self.hamiltonian = jordan_wigner(self.fermionic_op)
        elif mapper == 'bravyi_kitaev':
            self.hamiltonian = bravyi_kitaev(self.fermionic_op)
        
        self.n_qubits = self.hamiltonian.n_qubits
        
        print(f"Hamiltonian generated: {self.n_qubits} qubits")
        print(f"Number of terms: {len(self.hamiltonian.terms)}")
        
        return self.hamiltonian
    
    def _build_fermionic_hamiltonian(self, h1e, h2e):
        """
        Build fermionic Hamiltonian from molecular integrals
        """
        from openfermion import FermionOperator
        
        hamiltonian = FermionOperator()
        
        # One-electron terms
        for p in range(h1e.shape[0]):
            for q in range(h1e.shape[1]):
                if abs(h1e[p, q]) > 1e-12:
                    # Spin-up
                    hamiltonian += FermionOperator(f'{2*p}^ {2*q}', h1e[p, q])
                    # Spin-down  
                    hamiltonian += FermionOperator(f'{2*p+1}^ {2*q+1}', h1e[p, q])
        
        # Two-electron terms (implementation continues...)
        # [Additional fermionic operator construction code]
        
        return hamiltonian
    
    def analyze_hamiltonian(self):
        """
        Analyze Hamiltonian structure and properties
        """
        if self.hamiltonian is None:
            raise ValueError("Hamiltonian not generated yet")
        
        analysis = {
            'n_qubits': self.n_qubits,
            'n_terms': len(self.hamiltonian.terms),
            'max_weight': max([len(term) for term in self.hamiltonian.terms.keys()]),
            'coefficients': list(self.hamiltonian.terms.values())
        }
        
        # Analyze coefficient distribution
        coeffs = np.array([abs(c) for c in analysis['coefficients']])
        analysis['coeff_stats'] = {
            'mean': np.mean(coeffs),
            'std': np.std(coeffs),
            'max': np.max(coeffs),
            'min': np.min(coeffs[coeffs > 0])
        }
        
        return analysis

print("✅ MolecularHamiltonianBuilder class implemented")

In [ ]:
# Test with H2 molecule
print("Building H2 molecule Hamiltonian...")
h2_geometry = [['H', [0.0, 0.0, 0.0]], ['H', [0.0, 0.0, 0.74]]]

h2_builder = MolecularHamiltonianBuilder({'name': 'H2'})
h2_builder.build_molecule(h2_geometry)
h2_hamiltonian = h2_builder.generate_hamiltonian()

# Analyze H2 Hamiltonian
h2_analysis = h2_builder.analyze_hamiltonian()
print(f"\nH2 Hamiltonian Analysis:")
for key, value in h2_analysis.items():
    if key != 'coefficients':
        print(f"{key}: {value}")

## 2️⃣ Quantum Circuit Design for Molecular Systems 🔧

### 🎯 **Learning Focus:**
Hardware-efficient ansätze and quantum circuit architectures for molecular simulations

In [ ]:
class QuantumCircuitDesigner:
    """
    Design quantum circuits for molecular simulations
    """
    
    def __init__(self, n_qubits, n_electrons=None):
        self.n_qubits = n_qubits
        self.n_electrons = n_electrons or n_qubits // 2
        
    def hardware_efficient_ansatz(self, depth=1, entanglement='linear'):
        """
        Create hardware-efficient ansatz circuit
        """
        # Create parameters
        n_params = self.n_qubits * (depth + 1) + depth * self._count_entangling_gates(entanglement)
        params = ParameterVector('θ', n_params)
        
        qc = QuantumCircuit(self.n_qubits)
        param_idx = 0
        
        # Initial layer of single-qubit rotations
        for qubit in range(self.n_qubits):
            qc.ry(params[param_idx], qubit)
            param_idx += 1
        
        # Entangling layers
        for layer in range(depth):
            # Entangling gates
            if entanglement == 'linear':
                for qubit in range(self.n_qubits - 1):
                    qc.cx(qubit, qubit + 1)
            elif entanglement == 'circular':
                for qubit in range(self.n_qubits - 1):
                    qc.cx(qubit, qubit + 1)
                qc.cx(self.n_qubits - 1, 0)
            elif entanglement == 'full':
                for i in range(self.n_qubits):
                    for j in range(i + 1, self.n_qubits):
                        qc.cx(i, j)
            
            # Next layer of rotations
            for qubit in range(self.n_qubits):
                qc.ry(params[param_idx], qubit)
                param_idx += 1
        
        return qc, params
    
    def _count_entangling_gates(self, entanglement):
        """Count entangling gates for parameter calculation"""
        if entanglement == 'linear':
            return self.n_qubits - 1
        elif entanglement == 'circular':
            return self.n_qubits
        elif entanglement == 'full':
            return self.n_qubits * (self.n_qubits - 1) // 2
        return 0

print("✅ QuantumCircuitDesigner class implemented")

In [ ]:
# Test circuit design for H2
circuit_designer = QuantumCircuitDesigner(n_qubits=4, n_electrons=2)

# Create hardware-efficient ansatz
ansatz_circuit, params = circuit_designer.hardware_efficient_ansatz(depth=2, entanglement='linear')

print(f"Created ansatz circuit with {len(params)} parameters")
print(f"Circuit depth: {ansatz_circuit.depth()}")
print(f"Circuit gates: {ansatz_circuit.count_ops()}")

# Visualize circuit
print("\nCircuit structure:")
print(ansatz_circuit.draw(output='text', fold=-1))

## 📊 Module 1 Assessment & Checkpoint

### ✅ **Completion Checklist:**
- [ ] **Molecular Hamiltonian Builder** - H2 Hamiltonian successfully constructed
- [ ] **Quantum Circuit Designer** - Hardware-efficient ansatz implemented
- [ ] **Hamiltonian Analysis** - Structure and properties understood
- [ ] **Ready for VQE** - Foundation concepts mastered

### 🎯 **Knowledge Check:**
1. **What is the Jordan-Wigner mapping?** _____
2. **How many qubits are needed for H2?** _____
3. **What are the key components of a molecular Hamiltonian?** _____

### ⏭️ **Next Steps:**
**Ready to continue?** → [Day 6 Module 2: VQE & Molecular Ground States](day_06_module_2_vqe_algorithms.ipynb)

**Need more practice?** → Review molecular orbital theory and quantum circuit basics

**Struggling with concepts?** → [Community Support](https://github.com/yourusername/ChemML/discussions)

---

### 📈 **Progress Summary:**
**Module 1 Complete!** ✅  
**Time Investment:** _____ minutes  
**Mastery Level:** [ ] Beginner [ ] Intermediate [ ] Advanced  
**Confidence Score:** ___/10

---